data_loader에서 구현할 것

- 예상 시간 산출 기능 -> data_loader
- 데이터 적재 전체 과정 정의 후 구현 -> data_loader
- 세이브 포인트 기능 -> 일정 시간 단위로 가져올 때 마지막으로 가져왔던 위치를 안다면 가져오는 시간 절약 가능

데이터 적재(+그 후 넘겨주기까지) 과정

- 사용자가 분석 기간 입력 -> 구현 완료
- 해당 기간의 데이터를 '로컬 저장소'에서 불러오기 : DB가 되었건, 피클이건, ...
    - 만약 없다면? -> 이런 일 없도록 일정 시간 단위로 긁어오게 시킬 것
- 메모리에 데이터 적재
- prep할 수 있게 넘겨주기

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import os
import pandas as pd
import pickle
import re
import requests
from sys import stdout
import time

WRKON_JUPYTER = os.path.exists(os.path.abspath('../../resources'))

In [2]:
def get_user_input():
    start = input('분석 시작 지점을 예시와 같이 입력하시오 (ex. 2024년 1월 1일 -> 20240101)')
    end = input('분석 종료 지점을 예시와 같이 입력하시오 (ex. 2024년 1월 1일 -> 20240101)')

    print(
        f"\n\t분석 시작일 : {start[:4]}년 {start[4:6]}월 {start[6:]}일\n\t분석 종료일 : {end[:4]}년 {end[4:6]}월 {end[6:]}일\n"
    )
    is_proceed = input('맞으면 y, 틀리면 n을 입력하시오([y]/n)')
    
    if (is_proceed == 'y') or (is_proceed == '') :
        return start, end
    else:
        print("분석을 종료합니다\n\n")
        return None, None

In [3]:
def load_raw_data(start, end):
    
    fpath = '../../data/' if WRKON_JUPYTER else '../data/'
    
    raw_data_list = []
    
    period = pd.date_range(start, end).strftime('%Y%m%d')

    for date in period:
        raw_data = pd.read_csv(
            fpath+date+'.csv',
            col
        )
        raw_data['dates'] = date
        raw_data_list.append(raw_data)

    df = pd.concat(raw_data_list).reset_index(drop=True)

    return df

In [4]:
def get_title_cnt(date, sid1='001'):

    BASE_URL = f'https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1={sid1}&listType=title&'

    last_page = 1000
    n_titles = 0

    while True:
        url = BASE_URL
        url += f'date={date}&'
        url += f'page={last_page}'

        res = requests.get(url)
        bs = BeautifulSoup(res.text, features='html.parser')
        time.sleep(0.1)

        has_next = bs.find('a', class_='next nclicks(fls.page)')
        page_list = bs.find('div', class_='paging')
        
        if not has_next and last_page >= int(page_list.find('strong').get_text()):
            last_page = int(page_list.find('strong').get_text())
            n_titles = len(bs.find_all('a', class_="nclicks(fls.list)"))
            break
        else :
            last_page += 1000
    
    return (last_page - 1) * 50 + n_titles

In [5]:
def get_title_cnt_in_period(date_start, date_end, sid1='001'):
    
    title_cnt = pd.date_range(date_start, date_end).strftime("%Y%m%d").to_series()
    title_cnt = title_cnt.apply(get_title_cnt, sid1=sid1)
    title_cnt = title_cnt.sum()

    return title_cnt

In [6]:
def crawl(start, end, time_sleep=0.5, page_start=1) :
    BASE_URL = 'https://news.naver.com/main/list.naver?mode=LSD&mid=sec&listType=title&'

    period = pd.date_range(start, end).strftime('%Y%m%d')
    prompt = ''
    
    for date in period:
        
        raw_data = []
        max_page = get_title_cnt(date) // 50 + 1
        page = page_start
        pct = 0
        crawl_start = datetime.now()
        
        while True:            
            url = BASE_URL
            url += f'date={date}&'
            url += f'page={page}'
    
            res = requests.get(url)
            bs = BeautifulSoup(res.text, features='html.parser')
            time.sleep(time_sleep)

            raw_titles = [e.get_text() for e in bs.find_all('a', class_="nclicks(fls.list)")]
            raw_data.extend(raw_titles)

            pct = int(page * 100 / max_page)
            prompt = f'\r{date} ( MAX PAGE : {max_page} ) : '
            prompt += f'{page}, {pct}%'
            stdout.write(prompt)

            has_next = bs.find('a', class_='next nclicks(fls.page)')
            page_list = bs.find('div', class_='paging')
            
            if not has_next and int(page_list.find('strong').get_text()) >= max_page:
                break
            else : 
                page += 1

        prompt = '\tDONE '
        prompt += f'(start: {crawl_start}, '
        prompt += f'end: {datetime.now()}, '
        prompt += f'time: {datetime.now() - crawl_start}) / '
        
        stdout.write(prompt)

        if WRKON_JUPYTER : 
            save_raw_data(pd.Series(raw_data, name='titles'), fname=date+'.csv', fpath='../../data/')
        else : 
            save_raw_data(pd.Series(raw_data, name='titles'), fname=date+'.csv', fpath='../data/')

    return True

In [7]:
def save_raw_data(raw_data, fname, fpath):
    
    print('saving... ', end='')
    raw_data.to_csv(fpath+fname, index=False)
    print('DONE')

    return True

In [27]:
def convert_pkl_to_csv(fname):
    fpath = ''
    if WRKON_JUPYTER : 
        fpath='../../data/'
    else : 
        fpath='../data/'

    data = pd.read_pickle(fpath + fname)
    data = pd.Series(data['titles'], name='titles')
    data.to_csv(fpath+fname[:-4]+'.csv', index=False)
    return True

In [28]:
for date in ['20240101', '20240102', '20240103']:
    convert_pkl_to_csv(date + '.pkl')

0             [사설]尹 3년차도 “카르텔 타파”… 이젠 개혁과제 실행에 더 진력해야
1               일본서 한신 대지진보다 큰 규모 7.6 강진…"피해 현황 집계 중"
2                         동해안에도 지진해일 여파...인명·물적 피해 없어
3                            [라인W 날씨] 내일 오전까지 중부에 눈·비
4             성낙인 창녕군수 “2024년 청룡처럼 비상하는 창녕 되도록 노력할 것”
5            [사설]韓 2년 연속 무역적자, 새해 ‘반도체의 봄’만 기대하고 있기엔…
6    ‘관동대지진’ 100년만에...진도 1부터 7까지 일본 전역이 영향권[이시카와현 지진]
7      [신년사] 조규일 진주시장 "미래 성장동력 확보해 제3의 진주 기적 이뤄 나가겠다"
8                         평창 가스충전소 화재로 5명 중경상...주민 대피
9                      평창 가스 충전소 폭발...5명 다치고 인근 지역 정전
Name: titles, dtype: object
0        이수정 "이재명 습격범, 정신질환 아냐…나름의 목적 있을 것"
1      테슬라 작년 4분기 48만대 인도…中비야디 53만대에 뒤져(종합)
2             정부 “평창 LPG 충전소 폭발 재발방지 대책 추진”
3       [날씨] 내일 전국 곳곳에 눈·비…“빙판길 안전 운전 하세요!”
4       [사설]초유의 무허가 방송 사태 초래한 기형적 방통위 2인 체제
5            홍남표 창원시장,  새해 미래 50년 먹거리 사업 챙겨
6                 속초시, 올해 교육경비 보조금 18억 원 지원
7                  “내년 도입 예정 금융투자소득세 폐지 추진”
8               동해시, 지난해 고향사랑 기부금 목표액 초과 달성
9    [사설]초중고 1명당 교부금 8년 뒤 3천만원… 교육 재

In [19]:
import pickle

In [15]:
with open('../../data/20240101.pkl', 'rb') as f:
    p = pickle.load(f)

In [16]:
p

{'titles': ['[사설]尹 3년차도 “카르텔 타파”… 이젠 개혁과제 실행에 더 진력해야',
  '일본서 한신 대지진보다 큰 규모 7.6 강진…"피해 현황 집계 중"',
  '동해안에도 지진해일 여파...인명·물적 피해 없어',
  '[라인W 날씨] 내일 오전까지 중부에 눈·비',
  '성낙인 창녕군수 “2024년 청룡처럼 비상하는 창녕 되도록 노력할 것”',
  '[사설]韓 2년 연속 무역적자, 새해 ‘반도체의 봄’만 기대하고 있기엔…',
  '‘관동대지진’ 100년만에...진도 1부터 7까지 일본 전역이 영향권[이시카와현 지진]',
  '[신년사] 조규일 진주시장 "미래 성장동력 확보해 제3의 진주 기적 이뤄 나가겠다"',
  '평창 가스충전소 화재로 5명 중경상...주민 대피',
  '평창 가스 충전소 폭발...5명 다치고 인근 지역 정전',
  '밀양 분기점 부근 4중 추돌사고...1명 숨지고 3명 다쳐',
  'MIDEAST ISRAEL PALESTINIAN CONFLICT',
  'MIDEAST ISRAEL PALESTINIAN CONFLICT',
  '4중 추돌 사고 나자 20여 명 달려와 사고 수습 도와',
  '새만금 방조제 인근 텐트에서 50대 남녀 숨진 채 발견',
  '[횡설수설/우경임]1441일 만에 문 닫은 코로나 선별진료소',
  '[영상] 평창 LPG 충전소 가스 누출로 연쇄 폭발…블랙박스 영상',
  'GERMANY SKI JUMPING',
  'PAKISTAN HUMAN RIGHTS',
  'PAKISTAN HUMAN RIGHTS',
  '[오늘과 내일/장택동]말만 앞세운 空論으론 해결 못 할 재판 지연',
  '[간추린 뉴스] 월소득 213만 원 이하면 기초연금 수령 외',
  '[동아시론/박원곤]다중고에 직면한 김정은의 새해',
  '국민 절반 “소득 5만달러 중추국가 도약 눈앞”',
  '일본 기상청 "밤 11시 3분 이시카와현에 규모 4.6 지진 추가로 관측"',
  "[신년사] 이동환 고양시장 “경제자유구역·첨단기

In [25]:
load_raw_data('20240101', '20240103')

,0,dates
0,[사설]尹 3년차도 “카르텔 타파”… 이젠 개혁과제 실행에 더 진력해야,20240101
1,"일본서 한신 대지진보다 큰 규모 7.6 강진…""피해 현황 집계 중""",20240101
2,동해안에도 지진해일 여파...인명·물적 피해 없어,20240101
3,[라인W 날씨] 내일 오전까지 중부에 눈·비,20240101
4,성낙인 창녕군수 “2024년 청룡처럼 비상하는 창녕 되도록 노력할 것”,20240101
...,...,...
56145,"국무회의, 쌍특검법 거부권 의결 연기",20240103
56146,"김여정, 尹 신년사 저격…""안보 불안 일상사 된 건 윤 대통령 공로""",20240103
56147,"사우디, ‘러·중 주도’ 브릭스 회원국 가입 공식 발표",20240103
56148,"울산농협, 갑진년 시무식·3행(行) 3무(無) 실천 결의",20240103
